# Replication of results from Anderson et al. 2016

## Initializing

Notes :
- sampling frequency
- initial parameters (grid search, iterative)


In [1]:
import numpy as np
import scipy.io
import math
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import pyhsmm_mvpa as hsmm

import os

%load_ext autoreload
%autoreload 2


# Initial fit

Reading the data

In [9]:
mat = scipy.io.loadmat('matlab/newbumps/starter.mat')

data = np.array(mat['normedscore10'])
starts = np.array(mat['x'][:,0]) -1#correcting to be 0 indexed
ends = np.array(mat['y'][:,0])-1#correcting to be 0 indexed
subjects = np.array(mat['subjects'])-1 #correcting to be 0 indexed

testing single fit

In [10]:
init = hsmm.hsmm(data, starts, ends, sf=100, bump_width=50)

In [61]:
initial_estimates = init.fit_single(init.max_bumps())

Estimating parameters for 8 bumps model
(array([7430]),)


/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:286: RuntimeWarning: divide by zero encountered in log
  likelihood = np.sum(np.log(temp[:,:,0].sum(axis=0)))# why 0 index? PCG shouldn't it also be for all dim??
/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:288: RuntimeWarning: invalid value encountered in true_divide
  eventprobs = temp / np.tile(temp.sum(axis=0), [self.max_d, 1, 1])
/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:176: RuntimeWarning: invalid value encountered in double_scalars
  while (lkh - lkh1) > threshold:


UnboundLocalError: local variable 'magnitudes1' referenced before assignment

In [64]:
initial_estimates = init.fit_single(1)

Estimating parameters for 1 bumps model


/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:161: UserWarning: Using default parameters value for magnitudes and gamma
  parameters = np.tile([2, math.ceil(self.max_d)/(n_bumps+1)/2], (n_bumps+1,1))


(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
Parameters estimated for 1 bumps model


In [50]:
np.arange(3)[:-1]

array([0, 1])

In [65]:
initial_estimates

<xarray.Dataset>
Dimensions:      (stage: 2, params: 2, component: 10, bump: 1, samples: 300,
                  trial: 18271)
Dimensions without coordinates: stage, params, component, bump, samples, trial
Data variables:
    likelihoods  float64 -7.615e+04
    parameters   (stage, params) float64 2.0 11.49 2.0 41.13
    magnitudes   (component, bump) float64 0.8388 -1.232 ... -0.1847 0.01124
    eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [47]:
a=np.arange(1,28).reshape(3,3,3)

In [51]:
a.sum(axis=2)

array([[ 6, 15, 24],
       [33, 42, 51],
       [60, 69, 78]])

In [29]:
initial_estimates.bump

<xarray.DataArray 'bump' (bump: 7)>
array([0, 1, 2, 3, 4, 5, 6])
Dimensions without coordinates: bump

In [28]:
np.arange(np.max(initial_estimates.bump))

array([0, 1, 2, 3, 4, 5])

In [35]:
initial_estimates.magnitudes

<xarray.DataArray 'magnitudes' (component: 10, bump: 7)>
array([[-0.23720847,  0.8586386 ,  0.45124144, -0.32373778, -0.36754849,
        -0.28108556,  0.4004604 ],
       [ 0.1542926 , -1.11735336, -0.53047815,  0.22728637,  0.41498589,
         0.54907881,  0.4954078 ],
       [ 0.04559097,  0.02848087, -0.09265243, -0.12978779, -0.10614803,
        -0.09139781,  0.06974755],
       [ 0.08695621, -0.32578377,  0.48253903,  0.16689694, -0.27631524,
        -0.25358869,  0.33800248],
       [ 0.79578218, -0.87980019, -0.23811934,  0.11163558,  0.15644206,
        -0.13242173,  0.11258207],
       [-0.01135325,  0.40857113,  0.38330615,  0.18922912,  0.06114501,
         0.03692674, -0.32064054],
       [ 0.55771605, -0.69303947, -0.24877737, -0.12240566,  0.00824393,
        -0.14616817,  0.08345912],
       [-0.4512181 , -0.29017993, -0.29749777, -0.0091588 ,  0.19441473,
         0.33282112,  0.25973066],
       [-0.25646116, -0.16437636, -0.10860895,  0.02900031,  0.02718487,
         0.07898751,  0.34708263],
       [ 0.60546279,  0.06756241, -0.11922721, -0.41542628, -0.24751506,
        -0.03452771, -0.15226695]])
Dimensions without coordinates: component, bump

In [73]:
initial_estimates.parameters.sel(stage=[0,1])

<xarray.DataArray 'parameters' (stage: 2, params: 2)>
array([[2.        , 4.19538221],
       [2.        , 2.66145563]])
Dimensions without coordinates: stage, params

In [ ]:
initial_estimates.parameters

for n_bumps in np.arange(np.max(initial_estimates.bump)):
    for i in np.arange(n_bumps):
        init.calc_EEG_50h(initial_estimates.magnitudes.sel(bump=1).values, initial_estimates.parameters.sel(stage=[0,1]).values,1)
    

## Iterative fit

Parallel iterative fit

In [49]:
init = hsmm.hsmm(data, starts, ends, sf=100,bump_width = 50)

2
5


In [50]:
import multiprocessing 

cpus = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=cpus) as pool:
    results = pool.map(init.fit_single, np.arange(1,2))
    

Estimating parameters for 1 bumps model


/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:162: UserWarning: Using default parameters value for magnitudes and gamma
  magnitudes = np.zeros((self.n_dims,n_bumps))


Parameters estimated for 1 bumps model


In [51]:
results

[<xarray.Dataset>
 Dimensions:      (stage: 2, params: 2, component: 10, bump: 1, samples: 300,
                   trial: 18271)
 Dimensions without coordinates: stage, params, component, bump, samples, trial
 Data variables:
     likelihoods  float64 -8.765e+04
     parameters   (stage, params) float64 2.0 32.89 2.0 18.23
     magnitudes   (component, bump) float64 -1.237 0.347 ... 0.1337 -0.05722
     eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0]

results

Comparing with other bump width :

In [6]:
results_initial_bump_width = results

In [46]:
init = hsmm.hsmm(data, starts, ends, sf=100,bump_width = 40)

2
4


In [47]:
import multiprocessing 

cpus = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=cpus) as pool:
    results_70 = pool.map(init.fit_single, np.arange(1,3))
    

Estimating parameters for 1 bumps model


/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:162: UserWarning: Using default parameters value for magnitudes and gamma
  magnitudes = np.zeros((self.n_dims,n_bumps))


Estimating parameters for 2 bumps model


/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:162: UserWarning: Using default parameters value for magnitudes and gamma
  magnitudes = np.zeros((self.n_dims,n_bumps))


Parameters estimated for 1 bumps model
Parameters estimated for 2 bumps model


In [48]:
results_70

[<xarray.Dataset>
 Dimensions:      (stage: 2, params: 2, component: 10, bump: 1, samples: 300,
                   trial: 18271)
 Dimensions without coordinates: stage, params, component, bump, samples, trial
 Data variables:
     likelihoods  float64 -7.12e+04
     parameters   (stage, params) float64 2.0 11.4 2.0 39.72
     magnitudes   (component, bump) float64 0.8804 -1.327 ... -0.199 0.0009981
     eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0,
 <xarray.Dataset>
 Dimensions:      (stage: 3, params: 2, component: 10, bump: 2, samples: 300,
                   trial: 18271)
 Dimensions without coordinates: stage, params, component, bump, samples, trial
 Data variables:
     likelihoods  float64 -6.018e+04
     parameters   (stage, params) float64 2.0 10.51 2.0 21.87 2.0 16.99
     magnitudes   (component, bump) float64 0.9199 -1.235 ... 0.03953 -0.06129
     eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0]

Parallel iterative fit

### Downsampling test 

In [34]:
np.shape(data)

(1950396, 10)

In [39]:
down_sampled_data = data[0::2,:]

In [42]:
np.shape(down_sampled_data)

(975198, 10)

In [40]:
init_ds = hsmm.hsmm(down_sampled_data, starts, ends, sf=50 ,bump_width = 50)

[10. 30.]


In [ ]:
FIX START AND ENDS !!

In [41]:
import multiprocessing 

cpus = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=cpus) as pool:
    results_70 = pool.map(init_ds.fit_single, np.arange(1,8))
    

Estimating parameters for 1 bumps model
Estimating parameters for 2 bumps model
Estimating parameters for 3 bumps model
Estimating parameters for 4 bumps model
Estimating parameters for 5 bumps model
Estimating parameters for 6 bumps model
Estimating parameters for 7 bumps model


ValueError: could not broadcast input array from shape (17,1) into shape (59,1)